In [1]:
print("ok")

ok


In [2]:
'''
Only first time I need that
'''
# Import library
import gdown

# Gdrive link
gdrive_url = "https://drive.google.com/file/d/1y50gri4vFI3y1WZTEly1nlBA5pih-DPL/view?usp=sharing"

# ID of the file you want to download
file_id = gdrive_url.split('/')[-2]
file_id

'1y50gri4vFI3y1WZTEly1nlBA5pih-DPL'

In [3]:
# Download format
prefix = 'https://drive.google.com/uc?/export=download&id='

# Output you want
url = prefix+file_id
output = "../datasets/waste-material-dataset.zip"

In [4]:
def download_file(file_url, output):
    try:
        # Download the file
        gdown.download(file_url, output)
        print('File downloaded successfully.')
    except Exception as e:
        print('An error occurred:', e)

In [5]:
download_file(url, output)

Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1y50gri4vFI3y1WZTEly1nlBA5pih-DPL
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1y50gri4vFI3y1WZTEly1nlBA5pih-DPL&confirm=t&uuid=4751ca34-0048-4bf8-b8b5-f3186878d7ac
To: h:\Git_Projects\Waste-Material-System\src\backend\datasets\waste-material-dataset.zip
100%|██████████| 40.2M/40.2M [00:18<00:00, 2.12MB/s]

File downloaded successfully.


In [6]:
'''
Only first time I need that
'''
# Extract zip file
from zipfile import ZipFile

def extract_zip_files(file_name, unzip_path):
    try:
        with ZipFile(file_name, 'r') as file:
            print('Extract all the files...')
            file.extractall(path=unzip_path)
            print(f"Successfully extracted to {unzip_path}")
    except Exception as e:
        print(f"Extracting file error: {e}")

In [7]:
'''
Only first time I need that
'''
file_name = "../datasets/waste-material-dataset.zip"
unzip_path = "../datasets"

extract_zip_files(file_name, unzip_path)

Extract all the files...
Successfully extracted to ../datasets


# I developed several strategies to handle the dataset.

### 1. Comprehending the Dataset
### 2. EDA

In [ ]:
'''
Start here
'''
# Importing libraries
import numpy as np
import os, cv2, random
from collections import Counter, defaultdict
from tqdm import tqdm

# Data visualization libraries
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns

# Disable python warnings
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Load image and label directories
train_image_dir = "../datasets/data/train/images"
train_label_dir = "../datasets/data/train/labels"
valid_image_dir = "../datasets/data/valid/images"
valid_label_dir = "../datasets/data/valid/labels"
test_image_dir = "../datasets/data/test/images"
test_label_dir = "../datasets/data/test/labels"

In [ ]:
# Load yaml file
import yaml

with open("../datasets/data/data.yaml", "r") as f:
    dataset = yaml.load(f, Loader=yaml.FullLoader)

dataset

## 1. Understanding the Dataset

In [ ]:

def explain_dataset(image_dir, label_dir, class_names, num_classes):
    
    # Storage variables
    image_count = 0
    label_count = 0
    image_not_count = 0
    total_bboxes = 0

    images_per_class = defaultdict(int)
    instances_per_class = defaultdict(int)
    objects_per_image = defaultdict(int)

    missing_label_files = []
    empty_label_files = []
    images_without_annotations = []
    bbox_areas = []

    
    for img_file in os.listdir(image_dir):
        if not img_file.lower().endswith((".jpg", ".png", ".jpeg")):
            image_not_count += 1
            continue
        
        image_count += 1
        image_name = os.path.splitext(img_file)[0]
        label_file = os.path.join(label_dir, f"{image_name}.txt")
        
        # Check if label file exists
        if not os.path.exists(label_file):
            missing_label_files.append(img_file)
            images_without_annotations.append(img_file)
            continue
        else:
            label_count += 1
        
        # Read annotate 
        with open(label_file, "r") as f:
            lines = f.readlines()
        
        if lines == [''] or len(lines) == 0:
            empty_label_files.append(img_file)
            images_without_annotations.append(img_file)
            continue
        
        # Count objects
        objects_per_image[img_file] = len(lines)
        total_bboxes += len(lines)
        
        # Count instances
        for line in lines:
            cls, x, y, w, h = map(float, line.split())
            cls = int(cls)
            
            instances_per_class[cls] += 1
            images_per_class[cls] += 1
            
            # Calculate bbox area
            bbox_area = w * h
            bbox_areas.append(bbox_area)
        
    # DISPLAY STATISTICS
    print("\n====== DATASET STATISTICS ======\n")
    print("Total images:", image_count)
    print("Total label files:", label_count)
    print("Images other formats:", image_not_count)
    print("Total bounding boxes:", total_bboxes)
    print("Average bounding-box size (normalized area):", 
        np.mean(bbox_areas) if bbox_areas else 0)

    print("\n=== IMAGES PER CLASS ===")
    for cls in range(num_classes):
        print(f"{cls} ({class_names[cls]}): {images_per_class[cls]} images")

    print("\n=== INSTANCES PER CLASS ===")
    for cls in range(num_classes):
        print(f"{cls} ({class_names[cls]}): {instances_per_class[cls]} instances")
    
    print("\n=== OBJECTS PER IMAGE ===")
    avg_objects_per_img = sum(objects_per_image.values()) / len(objects_per_image)
    print(f"Average objects per image: {avg_objects_per_img}")

    print("\nImages without annotation labels:", len(images_without_annotations))
    print("Missing label files:", len(missing_label_files))
    print("Empty label files:", len(empty_label_files))

In [ ]:
class_names = dataset["names"]
num_classes = len(class_names)

explain_dataset(train_image_dir, train_label_dir, class_names, num_classes)

## Train Dataset Observations:

- I'm using YOLO format to understand and analyze the dataset.
- The dataset contains two splits: train and valid.
- Each split contains two folders:
    - images/ → original images
    - labels/ → YOLO annotation files
- Each annotation file contains lines in the format:
- class x_center y_center width height, all values normalized.

## Understanding Images and Annotations

- Each image may contain one or more annotations, but in this dataset most images contain exactly one object.
- Each annotation is linked to only one image through the same filename.
- Class IDs range from 0 to 12, mapped to 13 categories like "banana", "chilli", "drinkcan", etc.
- The dataset appears to have one instance per image (based on total images = 919 and total instances = 1180, with very few multi-object images).

## Parsed and Computed Statistics

- Loaded all images and their corresponding YOLO annotation files.

- Counted:
    - Total number of images
    - Total number of labels (same count → perfectly consistent)
    - Total number of bounding boxes (1180)
    - Bounding-box sizes
    - Objects per image
    - Instances per class
    - Images per class
- Verified data consistency:
    - No missing labels
    - No empty label files
    - No images without annotations
    - No images are any other format
- Bounding box statistics:
    - Average normalized bbox area = 0.3129
    (meaning boxes are generally large relative to image size)

## Class Distribution

- Extracted category IDs and counted occurrences of each class.
- The dataset has 13 object categories:
    - The largest class is sweetpotato (120 instances).
    - The smallest class is chilli (71 instances).
- Class distribution is relatively balanced, with no extremely rare classes.

## Counting Objects Per Image

- Computed number of objects per image across both splits.
- Most images contain a single object, since total instances (1180) is only slightly higher than total images (919).
- The average number of objects per image ≈ 1.28.

## Conclusions:

- The dataset contains two YOLO-format splits: train and valid.
- Directory structure is consistent, containing:
    - images/
    - labels/
    - data.yaml defining the 13 class names.
- All images have corresponding label files:
    - 0 missing labels
    - 0 empty labels
    - 0 unannotated images
    - 0 image are other format
    - This indicates a clean and well-prepared dataset.
- Each annotation follows YOLO format with class IDs and normalized bounding boxes.
- The dataset contains:
    - 919 images
    - 919 label files
    - 1180 object instances
- Class distribution is balanced overall:
    - Highest: sweetpotato (120)
    - Lowest: chilli (71)
- Bounding boxes are generally large, with an average normalized area of ~0.31.
- On average, each image contains 1.28 objects, meaning the dataset is primarily single-object per image.
- Category names match the 13 waste/food-related classes:
banana, chilli, drinkcan, drinkpack, foodcan, lettuce, paperbag, plasticbag, plasticbottle, plasticcontainer, sweetpotato, teabag, tissueroll.

In [ ]:
explain_dataset(valid_image_dir, valid_label_dir, class_names, num_classes)

## Valid Dataset Observations:

- I'm using YOLO format to understand and analyze the dataset.
- The dataset contains two splits: train and valid.
- Each split contains two folders:
    - images/ → original images
    - labels/ → YOLO annotation files
- Each annotation file contains lines in the format:
- class x_center y_center width height, all values normalized.

## Understanding Images and Annotations

- Each image may contain one or more annotations, but in this dataset most images contain exactly one object.
- Each annotation is linked to only one image through the same filename.
- Class IDs range from 0 to 12, mapped to 13 categories like "banana", "chilli", "drinkcan", etc.
- The dataset appears to have one instance per image (based on total images = 459 and total instances = 702, with very few multi-object images).

## Parsed and Computed Statistics

- Loaded all images and their corresponding YOLO annotation files.

- Counted:
    - Total number of images
    - Total number of labels (same count → perfectly consistent)
    - Total number of bounding boxes (702)
    - Bounding-box sizes
    - Objects per image
    - Instances per class
    - Images per class
- Verified data consistency:
    - No missing labels
    - No empty label files
    - No images without annotations
    - No images are any other format
- Bounding box statistics:
    - Average normalized bbox area = 0.2643
    (meaning boxes are generally large relative to image size)

## Class Distribution

- Extracted category IDs and counted occurrences of each class.
- The dataset has 13 object categories:
    - The largest class is tissueroll (63 instances).
    - The smallest class is plasticbag (40 instances).
- Class distribution is relatively balanced, with no extremely rare classes.

## Counting Objects Per Image

- Computed number of objects per image across both splits.
- Most images contain a single object, since total instances (702) is only slightly higher than total images (459).
- The average number of objects per image ≈ 1.52.

## Conclusions:

- The dataset contains two YOLO-format splits: train and valid.
- Directory structure is consistent, containing:
    - images/
    - labels/
    - data.yaml defining the 13 class names.
- All images have corresponding label files:
    - 0 missing labels
    - 0 empty labels
    - 0 unannotated images
    - 0 image are other format
    - This indicates a clean and well-prepared dataset.
- Each annotation follows YOLO format with class IDs and normalized bounding boxes.
- The dataset contains:
    - 459 images
    - 459 label files
    - 702 object instances
- Class distribution is balanced overall:
    - Highest: tissueroll (63)
    - Lowest: plasticbag (40)
- Bounding boxes are generally large, with an average normalized area of ~0.31.
- On average, each image contains 1.28 objects, meaning the dataset is primarily single-object per image.
- Category names match the 13 waste/food-related classes:
banana, chilli, drinkcan, drinkpack, foodcan, lettuce, paperbag, plasticbag, plasticbottle, plasticcontainer, sweetpotato, teabag, tissueroll.

### 1.1 Comperehend the dataset

In [ ]:
class_names = dataset["names"]
num_classes = len(class_names)

explain_dataset(train_image_dir, train_label_dir, class_names, num_classes)

## Train Dataset Observations:

- I'm using YOLO format to understand and analyze the dataset.
- The dataset contains three splits: train, valid, and test.
- Each split contains two folders:
    - images/ → original images
    - labels/ → YOLO annotation files
- Each annotation file contains lines in the format:
- class x_center y_center width height, all values normalized.

## Understanding Images and Annotations

- Each image may contain one or more annotations, but in this dataset most images contain exactly one object.
- Each annotation is linked to only one image through the same filename.
- Class IDs range from 0 to 12, mapped to 13 categories like "banana", "chilli", "drinkcan", etc.
- The dataset appears to have one instance per image (based on total images = 964 and total instances = 1235, with very few multi-object images).

## Parsed and Computed Statistics

- Loaded all images and their corresponding YOLO annotation files.

- Counted:
    - Total number of images
    - Total number of labels (same count → perfectly consistent)
    - Total number of bounding boxes (1235)
    - Bounding-box sizes
    - Objects per image
    - Instances per class
    - Images per class
- Verified data consistency:
    - No missing labels
    - No empty label files
    - No images without annotations
    - No images are any other format
- Bounding box statistics:
    - Average normalized bbox area = 0.3129
    (meaning boxes are generally large relative to image size)

## Class Distribution

- Extracted category IDs and counted occurrences of each class.
- The dataset has 13 object categories:
    - The largest class is sweetpotato (108 instances).
    - The smallest class is banana (78 instances).
- Class distribution is relatively balanced, with no extremely rare classes.

## Counting Objects Per Image

- Computed number of objects per image across both splits.
- Most images contain a single object, since total instances (1235) is only slightly higher than total images (964).
- The average number of objects per image ≈ 1.28.

## Conclusions:

- The dataset contains two YOLO-format splits: train and valid.
- Directory structure is consistent, containing:
    - images/
    - labels/
    - data.yaml defining the 13 class names.
- All images have corresponding label files:
    - 0 missing labels
    - 0 empty labels
    - 0 unannotated images
    - 0 image are other format
    - This indicates a clean and well-prepared dataset.
- Each annotation follows YOLO format with class IDs and normalized bounding boxes.
- The dataset contains:
    - 964 images
    - 964 label files
    - 1235 object instances
- Class distribution is balanced overall:
    - Highest: sweetpotato (108)
    - Lowest: banana (78)
- Bounding boxes are generally large, with an average normalized area of ~0.31.
- On average, each image contains 1.28 objects, meaning the dataset is primarily single-object per image.
- Category names match the 13 waste/food-related classes:
banana, chilli, drinkcan, drinkpack, foodcan, lettuce, paperbag, plasticbag, plasticbottle, plasticcontainer, sweetpotato, teabag, tissueroll.

In [ ]:
class_names = dataset["names"]
num_classes = len(class_names)

explain_dataset(valid_image_dir, valid_label_dir, class_names, num_classes)

## Valid Dataset Observations:

- I'm using YOLO format to understand and analyze the dataset.
- The dataset contains three splits: train, valid, and test.
- Each split contains two folders:
    - images/ → original images
    - labels/ → YOLO annotation files
- Each annotation file contains lines in the format:
- class x_center y_center width height, all values normalized.

## Understanding Images and Annotations

- Each image may contain one or more annotations, but in this dataset most images contain exactly one object.
- Each annotation is linked to only one image through the same filename.
- Class IDs range from 0 to 12, mapped to 13 categories like "banana", "chilli", "drinkcan", etc.
- The dataset appears to have one instance per image (based on total images = 275 and total instances = 408, with very few multi-object images).

## Parsed and Computed Statistics

- Loaded all images and their corresponding YOLO annotation files.

- Counted:
    - Total number of images
    - Total number of labels (same count → perfectly consistent)
    - Total number of bounding boxes (408)
    - Bounding-box sizes
    - Objects per image
    - Instances per class
    - Images per class
- Verified data consistency:
    - No missing labels
    - No empty label files
    - No images without annotations
    - No images are any other format
- Bounding box statistics:
    - Average normalized bbox area = 0.2745
    (meaning boxes are generally large relative to image size)

## Class Distribution

- Extracted category IDs and counted occurrences of each class.
- The dataset has 13 object categories:
    - The largest class is sweetpotato (52 instances).
    - The smallest class is chilli (19 instances).
- Class distribution is relatively balanced, with no extremely rare classes.

## Counting Objects Per Image

- Computed number of objects per image across both splits.
- Most images contain a single object, since total instances (408) is only slightly higher than total images (275).
- The average number of objects per image ≈ 1.48.

## Conclusions:

- The dataset contains two YOLO-format splits: train and valid.
- Directory structure is consistent, containing:
    - images/
    - labels/
    - data.yaml defining the 13 class names.
- All images have corresponding label files:
    - 0 missing labels
    - 0 empty labels
    - 0 unannotated images
    - 0 image are other format
    - This indicates a clean and well-prepared dataset.
- Each annotation follows YOLO format with class IDs and normalized bounding boxes.
- The dataset contains:
    - 275 images
    - 275 label files
    - 408 object instances
- Class distribution is balanced overall:
    - Highest: sweetpotato (52)
    - Lowest: chilli (19)
- Bounding boxes are generally large, with an average normalized area of ~0.2745.
- On average, each image contains 1.48 objects, meaning the dataset is primarily single-object per image.
- Category names match the 13 waste/food-related classes:
banana, chilli, drinkcan, drinkpack, foodcan, lettuce, paperbag, plasticbag, plasticbottle, plasticcontainer, sweetpotato, teabag, tissueroll.

In [ ]:
class_names = dataset["names"]
num_classes = len(class_names)

explain_dataset(test_image_dir, test_label_dir, class_names, num_classes)

## Test Dataset Observations:

- I'm using YOLO format to understand and analyze the dataset.
- The dataset contains three splits: train, valid, and test.
- Each split contains two folders:
    - images/ → original images
    - labels/ → YOLO annotation files
- Each annotation file contains lines in the format:
- class x_center y_center width height, all values normalized.

## Understanding Images and Annotations

- Each image may contain one or more annotations, but in this dataset most images contain exactly one object.
- Each annotation is linked to only one image through the same filename.
- Class IDs range from 0 to 12, mapped to 13 categories like "banana", "chilli", "drinkcan", etc.
- The dataset appears to have one instance per image (based on total images = 139 and total instances = 239, with very few multi-object images).

## Parsed and Computed Statistics

- Loaded all images and their corresponding YOLO annotation files.

- Counted:
    - Total number of images
    - Total number of labels (same count → perfectly consistent)
    - Total number of bounding boxes (239)
    - Bounding-box sizes
    - Objects per image
    - Instances per class
    - Images per class
- Verified data consistency:
    - No missing labels
    - No empty label files
    - No images without annotations
    - No images are any other format
- Bounding box statistics:
    - Average normalized bbox area = 0.2353
    (meaning boxes are generally large relative to image size)

## Class Distribution

- Extracted category IDs and counted occurrences of each class.
- The dataset has 13 object categories:
    - The largest class is sweetpotato (31 instances).
    - The smallest class is chilli, drinkcan, foodcan (15 instances).
- Class distribution is relatively balanced, with no extremely rare classes.

## Counting Objects Per Image

- Computed number of objects per image across both splits.
- Most images contain a single object, since total instances (239) is only slightly higher than total images (139).
- The average number of objects per image ≈ 1.7194

## Conclusions:

- The dataset contains two YOLO-format splits: train and valid.
- Directory structure is consistent, containing:
    - images/
    - labels/
    - data.yaml defining the 13 class names.
- All images have corresponding label files:
    - 0 missing labels
    - 0 empty labels
    - 0 unannotated images
    - 0 image are other format
    - This indicates a clean and well-prepared dataset.
- Each annotation follows YOLO format with class IDs and normalized bounding boxes.
- The dataset contains:
    - 139 images
    - 139 label files
    - 239 object instances
- Class distribution is balanced overall:
    - Highest: sweetpotato (31)
    - Lowest: chilli, foodcan, drinkcan (15)
- Bounding boxes are generally large, with an average normalized area of ~0.2353.
- On average, each image contains 1.7194 objects, meaning the dataset is primarily single-object per image.
- Category names match the 13 waste/food-related classes:
banana, chilli, drinkcan, drinkpack, foodcan, lettuce, paperbag, plasticbag, plasticbottle, plasticcontainer, sweetpotato, teabag, tissueroll.

## 2. EDA

In [ ]:
def object_class_distribution_and_percentage_analysis(class_names, instances_per_class, num_classes):
    plt.figure(figsize=(12,6))
    sns.barplot(x=list(class_names), y=[instances_per_class[i] for i in range(num_classes)], palette="viridis")
    plt.xticks(rotation=45)
    plt.title("Instances per Class")
    plt.show()

    # Percentage
    total_objects = sum(instances_per_class.values())
    percentages = {class_names[i]: (instances_per_class[i]/total_objects*100) for i in range(num_classes)}
    categoies = list(percentages.keys())
    values = list(percentages.values())

    plt.figure(figsize=(8,8))
    plt.pie(values, labels=categoies, autopct="%1.2f%%",  startangle=90, shadow=True, labeldistance=1.1, pctdistance=0.6)
    plt.title("Class Percentage Distribution")
    plt.show()

In [ ]:
def area_distribution_analysis(bbox_areas, bbox_widths, bbox_heights, bbox_centers):
    plt.figure(figsize=(8,5))
    sns.histplot(bbox_areas, bins=20, kde=True)
    plt.title("Bounding Box Area Distribution (normalized)")
    plt.show()

    # Width distribution
    plt.figure(figsize=(8,5))
    sns.histplot(bbox_widths, bins=20, kde=True)
    plt.title("Bounding Box Width Distribution")
    plt.show()

    # Height distribution
    plt.figure(figsize=(8,5))
    sns.histplot(bbox_heights, bins=20, kde=True)
    plt.title("Bounding Box Height Distribution")
    plt.show()

    # Heatmap of bbox centers
    xs = [c[0] for c in bbox_centers]
    ys = [c[1] for c in bbox_centers]
    plt.figure(figsize=(6,6))
    sns.kdeplot(x=xs, y=ys, fill=True, cmap="Reds")
    plt.title("Bounding Box Center Heatmap")
    plt.show()

In [ ]:
def image_resolution_analysis(image_info):
    widths = [w for _, w, _ in image_info]
    heights = [h for _, _, h in image_info]

    plt.figure(figsize=(8,6))
    plt.scatter(widths, heights, alpha=0.4)
    plt.xlabel("Width")
    plt.ylabel("Height")
    plt.title("Image Resolution Distribution")
    plt.show()
    
    plt.figure(figsize=(10, 6))
    plt.subplot(1, 2, 1)
    plt.hist(widths, bins=20, density=True)
    plt.xlabel("Width")
    plt.ylabel("Density")
    plt.title("Width Distribution")

    plt.subplot(1, 2, 2)
    plt.hist(heights, bins=20, density=True)
    plt.xlabel("Height")
    plt.ylabel("Density")
    plt.title("Height Distribution")
    plt.show()

    print("Average resolution:", (np.mean(widths), np.mean(heights)))

In [ ]:
# Plot bounding box area size distribution
def bbx_size(bbox_areas, aspect_ratios):
    plt.figure(figsize=(10, 6))
    plt.subplot(1, 2, 1)
    sns.histplot(bbox_areas, bins=20, kde=True)
    plt.title('Bounding Box Area Distribution')
    
    plt.subplot(1, 2, 2)
    sns.histplot(aspect_ratios, bins=20, kde=True)
    plt.title('Bounding Box Aspect Ratio Distribution')

    plt.tight_layout()
    plt.show()
    
    print(f"Bounding box area distribution: {np.percentile(bbox_areas, [25, 50, 75])}")
    print(f"Aspect ratio distribution: {np.percentile(aspect_ratios, [25, 50, 75])}")

In [ ]:
def outlier_analysis(bbox_areas):
    areas = np.array(bbox_areas)
    mean_area = np.mean(areas)
    std_area = np.std(areas)
    outliers = np.where((areas > mean_area + 3*std_area) | (areas < mean_area - 3*std_area))[0]

    print("\nBounding box outliers:", len(outliers))

    plt.figure(figsize=(8,5))
    sns.boxplot(x=areas)
    plt.title("Boxplot of Normalized BBox Area")
    plt.show()

In [ ]:
def co_occurrence_analysis(co_occurrence, class_names, num_classes):
    matrix = np.zeros((num_classes, num_classes), dtype=int)

    for c1 in co_occurrence:
        for c2 in co_occurrence[c1]:
            matrix[int(c1)][int(c2)] = co_occurrence[c1][c2]

    plt.figure(figsize=(12,10))
    sns.heatmap(matrix, xticklabels=class_names, yticklabels=class_names, cmap="Blues", annot=True)
    plt.title("Class Co-occurrence Heatmap")
    plt.show()

In [ ]:
def show_random_samples(annotations, n=3):
    samples = random.sample(annotations, n)
    
    for img_path, cls, xc, yc, bw, bh in samples:
        img = cv2.imread(img_path)
        if img is None:
            print(f"Failed to load: {img_path}")
            continue
        
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        print(f"Image format: {img.dtype}, Image size: {img.size}, Image shape: {img.shape}")
        
        h, w = img.shape[:2]
        
        # Numeric values
        xc, yc, bw, bh = map(float, (xc, yc, bw, bh))

        x1 = (xc - bw/2) * w
        y1 = (yc - bh/2) * h
        width = bw * w
        height = bh * h
        
        # clamp to image bounds
        x1 = max(0, x1)
        y1 = max(0, y1)
        x2 = min(w, x1 + width)
        y2 = min(h, y1 + height)
        
        width = x2 - x1
        height = y2 - y1
        
        # Draw everyhing on the same figure
        plt.figure(figsize=(6,6))
        ax = plt.gca()
        plt.imshow(img)
        
        rect = patches.Rectangle(
            (x1, y1), width, height, 
            linewidth=2, edgecolor='r', facecolor='none'
        )
        ax.add_patch(rect)
        
        plt.text(x1, max(0, y1-5), class_names[int(cls)], 
                 fontsize=12, color='r', bbox=dict(facecolor='yellow', alpha=0.5)
        )
        
        plt.axis("off")
        plt.show()

In [ ]:

def parse_dataset(image_dir, label_dir, class_names, num_classes):
    
    # Storage variables
    image_info = []
    annotations = []
    image_not_count = 0

    objects_per_image = Counter()
    instances_per_class = Counter()
    co_occurrence = defaultdict(lambda: Counter())

    missing_labels = []
    empty_annotations = []
    bbox_areas = []
    aspect_ratios = []
    bbox_widths = []
    bbox_heights = []
    bbox_centers = []
    
    for img_file in tqdm(os.listdir(image_dir)):
        if not img_file.lower().endswith((".jpg", ".png", ".jpeg")):
            image_not_count += 1
            continue
        
        img_path = os.path.join(image_dir, img_file)
        label_path = os.path.join(label_dir, img_file.rsplit(".",1)[0] + ".txt")
        
        # Load image resolution
        img = cv2.imread(img_path)
        
        if img is None:
            continue
        
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img_h, img_w = img.shape[:2]
        image_info.append((img_path, img_w, img_h))
        
        # Check annotations
        if not os.path.exists(label_path):
            missing_labels.append(img_path)
            continue
        
        with open(label_path, "r") as f:
            lines = [l.strip() for l in f.readlines() if l.strip()]
        
        if len(lines) == 0:
            empty_annotations.append(img_path)
            continue
        
        objects_per_image[img_path] = len(lines)
        
        # Track which classes in this image
        present_classes = set()
        
        for line in lines:
            cls, xc, yc, bw, bh = map(float, line.split())
            
            bbox_widths.append(bw)
            bbox_heights.append(bh)
            bbox_centers.append((xc, yc))
            
            bbox_area = bw * bh
            bbox_areas.append(bbox_area)
            
            aspect_ratio = bw / bh
            aspect_ratios.append(aspect_ratio)
            
            annotations.append((img_path, cls, xc, yc, bw, bh))
            present_classes.add(cls)
            instances_per_class[cls] += 1
            
            for c in present_classes:
                co_occurrence[int(cls)][int(c)] += 1
        
        for c1 in present_classes:
            for c2 in present_classes:
                if c1 != c2:
                    co_occurrence[c1][c2] += 1
    
    print("\n=== MISSING LABELS ===")
    print("Missing labels:", len(missing_labels))
    
    print("\n=== EMPTY ANNOTATIONS ===")
    print("Empty annotations:", len(empty_annotations))
    
    print("\n=== CLASS DISTRIBUTION ===")
    object_class_distribution_and_percentage_analysis(class_names, instances_per_class, num_classes)
    
    print("\n=== BOUNDING BOX ANALYSIS ===")
    area_distribution_analysis(bbox_areas, bbox_widths, bbox_heights, bbox_centers)
    
    print("\n=== IMAGE RESOLUTION ANALYSIS ===")
    image_resolution_analysis(image_info)
    
    print("\n=== AREA SIZE ===")
    bbx_size(bbox_areas, aspect_ratios)
    
    print("\n=== OUTLIERS ===")
    outlier_analysis(bbox_areas)
    
    print("\n=== CO-OCCURRENCE ===")
    co_occurrence_analysis(co_occurrence, class_names, num_classes)
    
    print("\n=== RANDOM SAMPLES ===")
    show_random_samples(annotations)

In [ ]:
# Valid dataset
class_names = dataset["names"]
num_classes = len(class_names)

parse_dataset(valid_image_dir, valid_label_dir, class_names, num_classes)

In [ ]:
# Counting objects per image
def img_count(image_dir, label_dir):
    objects_per_image = defaultdict(int)
    
    for img_file in os.listdir(image_dir):
        image_name = os.path.splitext(img_file)[0]
        label_file = os.path.join(label_dir, f"{image_name}.txt")
                
        # Read annotate 
        with open(label_file, "r") as f:
            lines = f.readlines()
        
        if lines == [''] or len(lines) == 0:
            continue
        
        # Count objects
        objects_per_image[img_file] = len(lines)
        
    return objects_per_image

In [ ]:
def plot_histogram(objects_per_count):
    sns.histplot(objects_per_count, kde=True)
    plt.title('Objects per Image Distribution')
    plt.show()

In [ ]:
def object_count_distribution(image_dir, label_dir):
    # Dataset object per image count
    data_object_count = img_count(image_dir, label_dir)
    print(f"Object per image: {Counter(data_object_count.values())}")
    object_count_values = list(data_object_count.values())

    plot_histogram(object_count_values)

    return data_object_count

In [ ]:
# Valid dataset object per image count

valid_object_count = object_count_distribution(valid_image_dir, valid_label_dir)

In [ ]:
def images_with_multiple_objects(object_count, category_id):
    data_multiple_objects = [image_id for image_id, count in object_count.items() if count == category_id]

    print(f"Images with {category_id} objects: {data_multiple_objects}")
    print(f"Images total lengths with multiple objects: {len(data_multiple_objects)}")

In [ ]:
# Valid dataset Identify images with 2 or more objects
images_with_multiple_objects(valid_object_count, 11)

## Valid Dataset Observations:

- Plot a graph of the 'Number of Objects' and 'Categories'. I've many categories of: 'Waste Material'.
    - The validation dataset contains 13 different object categories.
    - The distribution of objects across categories shows class imbalance, with some categories appearing more frequently than others.
- Plot a bounding box width and height distributions.
    - Bounding box width and height distributions indicate that most objects occupy a moderate-to-large portion of the image.
    - The histograms show the normalized bounding box width and height values.
- Plot an Image size distribution graph
    - All validation images have a fixed resolution of 640 × 480 pixels.
    - No variation in image size is observed across the validation dataset.
- Plot a graph of bounding box annotation visualization
    - We know about the images format, size, and shape: Image format: uint8, Image size: 921600, Image shape: (640, 480, 3)
    - Visualized samples confirm that bounding boxes are correctly aligned with objects.
    - Some images contain multiple bounding boxes, indicating the presence of more than one object per image.
- Plot a graph of bounding box area size and aspect ratio distribution
    - Bounding box area distribution shows that most objects have relatively large bounding box areas compared to the image size.
    - Aspect ratio distribution indicates that most bounding boxes are close to square.
    - The histograms of bounding box area and aspect ratio values are right-skewed.
- Plot a bounding box area size and aspect ratio distribution
    - Bounding box area distribution shows that most objects have relatively large bounding box areas compared to the image size.
    - Aspect ratio distribution indicates that most bounding boxes are close to square.
    - The histograms of bounding box area and aspect ratio values are right-skewed.
- Plot a graph of object per image count
    - The validation dataset contains both single-object and multi-object images.
    - Object-per-image count for the validation dataset: Object per image: Counter({1: 427, 12: 12, 2: 10, 11: 8, 13: 1, 10: 1})


## Conclusions:

- The validation dataset contains 13 object categories, covering various waste and food-related classes.
- The number of objects per category is imbalanced, which may influence validation performance across different classes.
- Bounding box width, height, and area distributions indicate that objects are generally moderate to large in size, simplifying object localization compared to datasets dominated by small objects.
- All images have a uniform resolution of 640 × 480 pixels, which ensures consistent input dimensions during model validation.
- Bounding box annotation visualizations confirm that annotations are accurate and properly aligned with the objects in the images.
- Bounding box area and aspect ratio distributions show that most objects are square-shaped, with few extreme aspect ratios.
- Object-per-image analysis shows that many validation images contain multiple object instances, increasing scene complexity.
- The presence of multiple objects and class imbalance in the validation dataset may impact validation metrics, particularly for less frequent categories.
- Overall, the validation dataset is clean, well-annotated, and suitable for evaluating YOLO-based object detection models, while requiring careful interpretation of results due to class imbalance and varying object counts per image.
- Category names match the 13 waste/food-related classes:
banana, chilli, drinkcan, drinkpack, foodcan, lettuce, paperbag, plasticbag, plasticbottle, plasticcontainer, sweetpotato, teabag, tissueroll.

In [ ]:
# Train dataset
class_names = dataset["names"]
num_classes = len(class_names)

parse_dataset(train_image_dir, train_label_dir, class_names, num_classes)

In [ ]:
# Train dataset object per image count

train_object_count = object_count_distribution(train_image_dir, train_label_dir)

In [ ]:
# Valid dataset Identify images with 2 or more objects
images_with_multiple_objects(train_object_count, 13)

## Train Dataset Observations:

- Plot a graph of the 'Number of Objects' and 'Categories'. I've many categories of: 'Waste Material'.
    - The train dataset contains 13 different object categories.
    - The distribution of objects across categories shows class imbalance, with some categories appearing more frequently than others.
- Plot a bounding box width and height distributions.
    - Bounding box width and height distributions indicate that most objects occupy a moderate-to-large portion of the image.
    - The histograms show the normalized bounding box width and height values.
- Plot an Image size distribution graph
    - All train images have a fixed resolution of 640 × 480 pixels.
    - No variation in image size is observed across the train dataset.
- Plot a graph of bounding box annotation visualization
    - We know about the images format, size, and shape: Image format: uint8, Image size: 921600, Image shape: (640, 480, 3)
    - Visualized samples confirm that bounding boxes are correctly aligned with objects.
    - Some images contain multiple bounding boxes, indicating the presence of more than one object per image.
- Plot a graph of bounding box area size and aspect ratio distribution
    - Bounding box area distribution shows that most objects have relatively large bounding box areas compared to the image size.
    - Aspect ratio distribution indicates that most bounding boxes are close to square.
    - The histograms of bounding box area and aspect ratio values are right-skewed.
- Plot a bounding box area size and aspect ratio distribution
    - Bounding box area distribution shows that most objects have relatively large bounding box areas compared to the image size.
    - Aspect ratio distribution indicates that most bounding boxes are close to square.
    - The histograms of bounding box area and aspect ratio values are right-skewed.
- Plot a graph of object per image count
    - The train dataset contains both single-object and multi-object images.
    - Object-per-image count for the train dataset: Object per image: Counter({1: 880, 2: 16, 12: 11, 11: 7, 13: 3, 10: 2})


## Conclusions:

- The train dataset contains 13 object categories, covering various waste and food-related classes.
- The number of objects per category is imbalanced, which may influence train performance across different classes.
- Bounding box width, height, and area distributions indicate that objects are generally moderate to large in size, simplifying object localization compared to datasets dominated by small objects.
- All images have a uniform resolution of 640 × 480 pixels, which ensures consistent input dimensions during model train.
- Bounding box annotation visualizations confirm that annotations are accurate and properly aligned with the objects in the images.
- Bounding box area and aspect ratio distributions show that most objects are square-shaped, with few extreme aspect ratios.
- Object-per-image analysis shows that many train images contain multiple object instances, increasing scene complexity.
- The presence of multiple objects and class imbalance in the train dataset may impact train metrics, particularly for less frequent categories.
- Overall, the train dataset is clean, well-annotated, and suitable for evaluating YOLO-based object detection models, while requiring careful interpretation of results due to class imbalance and varying object counts per image.
- Category names match the 13 waste/food-related classes:
banana, chilli, drinkcan, drinkpack, foodcan, lettuce, paperbag, plasticbag, plasticbottle, plasticcontainer, sweetpotato, teabag, tissueroll.

### 2.2 EDA

In [ ]:
# Train dataset
class_names = dataset["names"]
num_classes = len(class_names)

parse_dataset(train_image_dir, train_label_dir, class_names, num_classes)

In [ ]:
# Train dataset object per image count

train_object_count = object_count_distribution(train_image_dir, train_label_dir)

In [ ]:
# Train dataset Identify images with 2 or more objects
images_with_multiple_objects(train_object_count, 13)

## Train Dataset Observations:

- Plot a graph of the 'Number of Objects' and 'Categories'. I've many categories of: 'Waste Material'.
    - The train dataset contains 13 different object categories.
    - The distribution of objects across categories shows class imbalance, with some categories appearing more frequently than others.
- Plot a bounding box width and height distributions.
    - Bounding box width and height distributions indicate that most objects occupy a moderate-to-large portion of the image.
    - The histograms show the normalized bounding box width and height values.
- Plot an Image size distribution graph
    - All train images have a fixed resolution of 640 × 480 pixels.
    - No variation in image size is observed across the train dataset.
- Plot a graph of bounding box annotation visualization
    - We know about the images format, size, and shape: Image format: uint8, Image size: 921600, Image shape: (640, 480, 3)
    - Visualized samples confirm that bounding boxes are correctly aligned with objects.
    - Some images contain multiple bounding boxes, indicating the presence of more than one object per image.
- Plot a graph of bounding box area size and aspect ratio distribution
    - Bounding box area distribution shows that most objects have relatively large bounding box areas compared to the image size.
    - Aspect ratio distribution indicates that most bounding boxes are close to square.
    - The histograms of bounding box area and aspect ratio values are right-skewed.
- Plot a bounding box area size and aspect ratio distribution
    - Bounding box area distribution shows that most objects have relatively large bounding box areas compared to the image size.
    - Aspect ratio distribution indicates that most bounding boxes are close to square.
    - The histograms of bounding box area and aspect ratio values are right-skewed.
- Plot a graph of object per image count
    - The train dataset contains both single-object and multi-object images.
    - Object-per-image count for the train dataset: Object per image: Counter({1: 923, 2: 17, 12: 12, 11: 8, 13: 2, 10: 2})


## Conclusions:

- The train dataset contains 13 object categories, covering various waste and food-related classes.
- The number of objects per category is imbalanced, which may influence train performance across different classes.
- Bounding box width, height, and area distributions indicate that objects are generally moderate to large in size, simplifying object localization compared to datasets dominated by small objects.
- All images have a uniform resolution of 640 × 480 pixels, which ensures consistent input dimensions during model train.
- Bounding box annotation visualizations confirm that annotations are accurate and properly aligned with the objects in the images.
- Bounding box area and aspect ratio distributions show that most objects are square-shaped, with few extreme aspect ratios.
- Object-per-image analysis shows that many train images contain multiple object instances, increasing scene complexity.
- The presence of multiple objects and class imbalance in the train dataset may impact train metrics, particularly for less frequent categories.
- Overall, the train dataset is clean, well-annotated, and suitable for evaluating YOLO-based object detection models, while requiring careful interpretation of results due to class imbalance and varying object counts per image.
- Category names match the 13 waste/food-related classes:
banana, chilli, drinkcan, drinkpack, foodcan, lettuce, paperbag, plasticbag, plasticbottle, plasticcontainer, sweetpotato, teabag, tissueroll.

In [ ]:
# Valid dataset
class_names = dataset["names"]
num_classes = len(class_names)

parse_dataset(valid_image_dir, valid_label_dir, class_names, num_classes)

In [ ]:
# Valid dataset object per image count

valid_object_count = object_count_distribution(valid_image_dir, valid_label_dir)

In [ ]:
# Valid dataset Identify images with 2 or more objects
images_with_multiple_objects(valid_object_count, 11)

## Valid Dataset Observations:

- Plot a graph of the 'Number of Objects' and 'Categories'. I've many categories of: 'Waste Material'.
    - The validation dataset contains 13 different object categories.
    - The distribution of objects across categories shows class imbalance, with some categories appearing more frequently than others.
- Plot a bounding box width and height distributions.
    - Bounding box width and height distributions indicate that most objects occupy a moderate-to-large portion of the image.
    - The histograms show the normalized bounding box width and height values.
- Plot an Image size distribution graph
    - All validation images have a fixed resolution of 640 × 480 pixels.
    - No variation in image size is observed across the validation dataset.
- Plot a graph of bounding box annotation visualization
    - We know about the images format, size, and shape: Image format: uint8, Image size: 921600, Image shape: (640, 480, 3)
    - Visualized samples confirm that bounding boxes are correctly aligned with objects.
    - Some images contain multiple bounding boxes, indicating the presence of more than one object per image.
- Plot a graph of bounding box area size and aspect ratio distribution
    - Bounding box area distribution shows that most objects have relatively large bounding box areas compared to the image size.
    - Aspect ratio distribution indicates that most bounding boxes are close to square.
    - The histograms of bounding box area and aspect ratio values are right-skewed.
- Plot a bounding box area size and aspect ratio distribution
    - Bounding box area distribution shows that most objects have relatively large bounding box areas compared to the image size.
    - Aspect ratio distribution indicates that most bounding boxes are close to square.
    - The histograms of bounding box area and aspect ratio values are right-skewed.
- Plot a graph of object per image count
    - The validation dataset contains both single-object and multi-object images.
    - Object-per-image count for the validation dataset: Object per image: Counter({1: 257, 2: 6, 12: 6, 11: 4, 13: 1, 10: 1})


## Conclusions:

- The validation dataset contains 13 object categories, covering various waste and food-related classes.
- The number of objects per category is imbalanced, which may influence validation performance across different classes.
- Bounding box width, height, and area distributions indicate that objects are generally moderate to large in size, simplifying object localization compared to datasets dominated by small objects.
- All images have a uniform resolution of 640 × 480 pixels, which ensures consistent input dimensions during model validation.
- Bounding box annotation visualizations confirm that annotations are accurate and properly aligned with the objects in the images.
- Bounding box area and aspect ratio distributions show that most objects are square-shaped, with few extreme aspect ratios.
- Object-per-image analysis shows that many validation images contain multiple object instances, increasing scene complexity.
- The presence of multiple objects and class imbalance in the validation dataset may impact validation metrics, particularly for less frequent categories.
- Overall, the validation dataset is clean, well-annotated, and suitable for evaluating YOLO-based object detection models, while requiring careful interpretation of results due to class imbalance and varying object counts per image.
- Category names match the 13 waste/food-related classes:
banana, chilli, drinkcan, drinkpack, foodcan, lettuce, paperbag, plasticbag, plasticbottle, plasticcontainer, sweetpotato, teabag, tissueroll.

In [ ]:
# Test dataset
class_names = dataset["names"]
num_classes = len(class_names)

parse_dataset(test_image_dir, test_label_dir, class_names, num_classes)

In [ ]:
# Test dataset object per image count

test_object_count = object_count_distribution(test_image_dir, test_label_dir)

In [ ]:
# Test dataset Identify images with 2 or more objects
images_with_multiple_objects(test_object_count, 2)

## Test Dataset Observations:

- Plot a graph of the 'Number of Objects' and 'Categories'. I've many categories of: 'Waste Material'.
    - The test dataset contains 13 different object categories.
    - The distribution of objects across categories shows class imbalance, with some categories appearing more frequently than others.
- Plot a bounding box width and height distributions.
    - Bounding box width and height distributions indicate that most objects occupy a moderate-to-large portion of the image.
    - The histograms show the normalized bounding box width and height values.
- Plot an Image size distribution graph
    - All test images have a fixed resolution of 640 × 480 pixels.
    - No variation in image size is observed across the test dataset.
- Plot a graph of bounding box annotation visualization
    - We know about the images format, size, and shape: Image format: uint8, Image size: 921600, Image shape: (640, 480, 3)
    - Visualized samples confirm that bounding boxes are correctly aligned with objects.
    - Some images contain multiple bounding boxes, indicating the presence of more than one object per image.
- Plot a graph of bounding box area size and aspect ratio distribution
    - Bounding box area distribution shows that most objects have relatively large bounding box areas compared to the image size.
    - Aspect ratio distribution indicates that most bounding boxes are close to square.
    - The histograms of bounding box area and aspect ratio values are right-skewed.
- Plot a bounding box area size and aspect ratio distribution
    - Bounding box area distribution shows that most objects have relatively large bounding box areas compared to the image size.
    - Aspect ratio distribution indicates that most bounding boxes are close to square.
    - The histograms of bounding box area and aspect ratio values are right-skewed.
- Plot a graph of object per image count
    - The test dataset contains both single-object and multi-object images.
    - Object-per-image count for the test dataset: Object per image: Counter({1: 127, 12: 5, 2: 3, 11: 3, 13: 1})


## Conclusions:

- The test dataset contains 13 object categories, covering various waste and food-related classes.
- The number of objects per category is imbalanced, which may influence test performance across different classes.
- Bounding box width, height, and area distributions indicate that objects are generally moderate to large in size, simplifying object localization compared to datasets dominated by small objects.
- All images have a uniform resolution of 640 × 480 pixels, which ensures consistent input dimensions during model test.
- Bounding box annotation visualizations confirm that annotations are accurate and properly aligned with the objects in the images.
- Bounding box area and aspect ratio distributions show that most objects are square-shaped, with few extreme aspect ratios.
- Object-per-image analysis shows that many test images contain multiple object instances, increasing scene complexity.
- The presence of multiple objects and class imbalance in the test dataset may impact test metrics, particularly for less frequent categories.
- Overall, the test dataset is clean, well-annotated, and suitable for evaluating YOLO-based object detection models, while requiring careful interpretation of results due to class imbalance and varying object counts per image.
- Category names match the 13 waste/food-related classes:
banana, chilli, drinkcan, drinkpack, foodcan, lettuce, paperbag, plasticbag, plasticbottle, plasticcontainer, sweetpotato, teabag, tissueroll.

In [ ]:
# Plot dataset split
dataset_size = [len(os.listdir(train_image_dir)), len(os.listdir(valid_image_dir)), len(os.listdir(test_image_dir))]
plt.figure(figsize=(10,6))

plt.subplot(1, 2, 1)
sns.barplot(x=["Train", "Valid", "Test"], y=dataset_size)
plt.xlabel("Dataset")
plt.ylabel("Number of Images")
plt.title("Dataset Split")

plt.subplot(1, 2, 2)
plt.pie(dataset_size, labels=["Train", "Valid", "Test"], autopct="%1.1f%%")
plt.title('Dataset Split Percentage')
plt.tight_layout()
plt.show()

In [ ]:
# Find train and valid duplicate images

train_image_files = os.listdir(train_image_dir)
valid_image_files = os.listdir(valid_image_dir)

train_and_valid_duplicate_images = [file for file in train_image_files if file in valid_image_files]
print(f"Duplicate images: {train_and_valid_duplicate_images}")
print(f"Duplicate images total length: {len(train_and_valid_duplicate_images)}")

In [ ]:
# Find train and test duplicate images

train_image_files = os.listdir(train_image_dir)
test_image_files = os.listdir(test_image_dir)

train_and_test_duplicate_images = [file for file in train_image_files if file in test_image_files]
print(f"Duplicate images: {train_and_test_duplicate_images}")
print(f"Duplicate images total length: {len(train_and_test_duplicate_images)}")

In [ ]:
# Find test and valid duplicate images

test_image_files = os.listdir(test_image_dir)
valid_image_files = os.listdir(valid_image_dir)

test_and_valid_duplicate_images = [file for file in test_image_files if file in valid_image_files]
print(f"Duplicate images: {test_and_valid_duplicate_images}")
print(f"Duplicate images total length: {len(test_and_valid_duplicate_images)}")

In [ ]:
# Find train and valid duplicate labels

train_label_files = os.listdir(train_label_dir)
valid_label_files = os.listdir(valid_label_dir)

train_and_valid_duplicate_labels = [file for file in train_label_files if file in valid_label_files]
print(f"Duplicate labels: {train_and_valid_duplicate_labels}")
print(f"Duplicate labels total length: {len(train_and_valid_duplicate_labels)}")

In [ ]:
# Find train and test duplicate labels

train_label_files = os.listdir(train_label_dir)
test_label_files = os.listdir(test_label_dir)

train_and_test_duplicate_labels = [file for file in train_label_files if file in test_label_files]
print(f"Duplicate labels: {train_and_test_duplicate_labels}")
print(f"Duplicate labels total length: {len(train_and_test_duplicate_labels)}")

In [ ]:
# Find test and valid duplicate labels

test_label_files = os.listdir(test_label_dir)
valid_label_files = os.listdir(valid_label_dir)

test_and_valid_duplicate_labels = [file for file in test_label_files if file in valid_label_files]
print(f"Duplicate labels: {test_and_valid_duplicate_labels}")
print(f"Duplicate labels total length: {len(test_and_valid_duplicate_labels)}")